In [ ]:
import pandas as pd
import numpy as np
import tkinter as tk
import warnings
import locale
import calendar
import chardet
import os
from tkinter import messagebox
from datetime import datetime
from unidecode import unidecode

warnings.filterwarnings('ignore')
locale.setlocale(locale.LC_TIME, 'es_ES.UTF-8')

rj_path = 'bases/gestiones/RJ'
clasa_path = 'bases/gestiones/CLASA'
mornese_path = 'bases/gestiones/MORNESE'

rj_archivos = [file for file in os.listdir(rj_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.txt')]
clasa_archivos = [file for file in os.listdir(clasa_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.txt')]
mornese_archivos = [file for file in os.listdir(mornese_path) if file.startswith('REPORTE_CORTES_BBVA_') and file.endswith('.txt')]

print(rj_archivos)
print(clasa_archivos)
print(mornese_archivos)

In [16]:
def clean_columns(columns_list: list[str]) -> list[str]:
    return [column.strip().replace('.', '').replace(' ', '_').upper() for column in columns_list]

In [3]:
def detectar_codificacion(ruta_archivo):
    with open(ruta_archivo, 'rb') as f:
        resultado = chardet.detect(f.read())
    return resultado['encoding']

path_tmp = 'bases/gestiones/RJ/REPORTE_CORTES_BBVA_2412.txt'
codificacion = detectar_codificacion(path_tmp)
codificacion

In [6]:
def clean_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    if 'AGENCIA' in df.columns:
        df['AGENCIA'] = df['AGENCIA'].str.upper().replace('ASESCOM RJ ABOGADOS', 'ASESCOM RJ')
    if 'TIPO_DE_MARCACIÓN' in df.columns:
        df.rename(columns={'TIPO_DE_MARCACIÓN': 'TIPO_MARCADOR'}, inplace=True)
    if 'TIPO_DE_DISCADO' in df.columns:
        df.rename(columns={'TIPO_DE_DISCADO': 'TIPO_MARCADOR'}, inplace=True)
    if 'PRIORIDAD_TELEFONO' not in df.columns:
        df['PRIORIDAD_TELEFONO'] = np.nan
    
    df['MARCA'] = df['MARCA'].str.strip().str.upper().replace(' ', '', regex=True)
    df['MARCA'] = np.where(df['MARCA'] == 'KST', 'KSTBC', df['MARCA'])
    df = df[df['MARCA'].isin(['BC', 'EXJ', 'KSTBC'])]
    df['FECHA_LLAMADA'] = pd.to_datetime(df['FECHA_LLAMADA'], format='%d/%m/%Y', errors='coerce')
    df['HORA_INICIO_LLAMADA'] = pd.to_datetime(df['HORA_INICIO_LLAMADA']).dt.hour.astype('Int64').fillna(0)
    df['TIPO_CONTACTO'] = df['TIPO_CONTACTO'].str.upper().replace('TERCERO', 'CONTACTO INDIRECTO').replace('CONTACTO EFECTIVO', 'CONTACTO DIRECTO').replace('EFECTIVO', 'CONTACTO DIRECTO')
    df['TRAMO'] = df['TRAMO'].str.strip().str.upper().replace(' ', '', regex=True).replace(['T1', 'T2', 'T3', 'T4'], 'NULL')
    df['TRAMO'] = df['TRAMO'].fillna('NULL')
    df['TIPO_MARCADOR'] = df['TIPO_MARCADOR'].str.strip().str.upper().replace(' ', '', regex=True)
    df['TIPO_MARCADOR'] = df['TIPO_MARCADOR'].fillna('NULL')
    df['PDP'] = df.apply(lambda x: 1 if pd.notna(x['FECHA_PDP']) else 0, axis=1)
    
    df.sort_values(by='FECHA_LLAMADA', inplace=True)
    df.reset_index(inplace=True, drop=True)
    df.drop(columns={'HORA_INICIO_LLAMADA'}, inplace=True)
    
    df = df[['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'MARCA', 'FECHA_LLAMADA', 'TIPO_CONTACTO', 'TRAMO', 'PDP', 'TIPO_MARCADOR', 'PRIORIDAD_TELEFONO']]
    return df

In [7]:
def consolidar_archivos(agencia_path: str, archivos: list) -> pd.DataFrame:
    dataframes = []
    for archivo in archivos:
        ruta_completa = os.path.join(agencia_path, archivo)
        df = pd.read_csv(
            ruta_completa, 
            sep='|', 
            encoding='UTF-8',
            index_col=False,
            dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
            )
        
        df.columns = clean_columns(df.columns)
        print(df.shape, df.columns)
        dataframes.append(df)
        print(f'{archivo} leído correctamente\n')
    
    df_consolidado = pd.concat(dataframes, ignore_index=True)
    print(df_consolidado.shape)
    df_consolidado = clean_dataframe(df_consolidado)
    
    df_consolidado.to_csv(
        os.path.join(agencia_path, f'CONSOLIDADO_GESTIONES_{agencia_path.split("/")[-1]}.txt'), 
        index=False, 
        sep='|',
        encoding='UTF-8',
    )
    return df_consolidado

In [8]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Consolidar archivos de gestiones?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        'RJ': (rj_path, rj_archivos),
        #'CLASA': (clasa_path, clasa_archivos),
        #'MORNESE': (mornese_path, mornese_archivos)
    }
    
    for key, (path, file_list) in archivos.items():
        if file_list:
            dataframes[key] = consolidar_archivos(path, file_list)
            print(f'Consolidado {key}:', dataframes[key].shape)
        else:
            print(f'No hay archivos para consolidar en {key}.')

root.destroy()

In [ ]:
root = tk.Tk()
root.attributes('-topmost', True)
root.withdraw()

result = messagebox.askquestion('Confirmación', '¿Leer consolidados?', icon='warning')

if result == 'yes':
    dataframes = {}
    archivos = {
        'RJ': f'{rj_path}/CONSOLIDADO_GESTIONES_RJ.txt',
        #'CLASA': f'{clasa_path}/CONSOLIDADO_GESTIONES_CLASA.txt',
        'MORNESE': f'{mornese_path}/CONSOLIDADO_GESTIONES_MORNESE.txt'
    }
    
    for key, file_path in archivos.items():
        if os.path.exists(file_path):
            dataframes[key] = pd.read_csv(
                file_path, 
                sep='|', 
                dtype={'CONTRATO': str, 'COD_CENTRAL': str, 'NRO_DOC': str}, 
                encoding='UTF-8'
                )
            print(f'Consolidado {key} leído:', dataframes[key].shape)
        else:
            print(f'El archivo CONSOLIDADO_GESTIONES_{key}.txt no existe.')

root.destroy()

In [ ]:
def sort_columns(df: pd.DataFrame) -> pd.DataFrame:
    cols = ['AGENCIA', 'COD_CENTRAL', 'CONTRATO', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'TIPO_CONTACTO', 'PRIORIDAD_TELEFONO', 
            'TIPO_MARCADOR', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'COBERTURA', 'CONTACTO_DIRECTO', 'PDP']
    return df[cols]

df_test: pd.DataFrame = dataframes['RJ'].copy()
df_test['TRAMO'] = df_test['TRAMO'].fillna('NULL')

print(df_test.columns)
print(df_test.shape)
df_test.head(5)

In [ ]:
df_test['FECHA_LLAMADA'].value_counts(dropna=False)

In [ ]:
df_test['PERIODO'] = pd.to_datetime(df_test['FECHA_LLAMADA']).dt.strftime('%Y%m').astype(str)

fechas_unicas = df_test[['PERIODO', 'FECHA_LLAMADA']].drop_duplicates().sort_values(by='FECHA_LLAMADA')
orden_fechas = fechas_unicas.groupby('PERIODO').cumcount() + 1
fechas_unicas['DIA'] = orden_fechas
df_test = df_test.merge(fechas_unicas[['PERIODO', 'FECHA_LLAMADA', 'DIA']], on=['PERIODO', 'FECHA_LLAMADA'], how='left')
df_test.drop(columns={'FECHA_LLAMADA'}, inplace=True)

df_test['INTENSIDAD_TOTAL'] = 1
df_test['INTENSIDAD_DIRECTA'] = np.where(df_test['TIPO_CONTACTO'] == 'CONTACTO DIRECTO', 1, 0)
df_test['COBERTURA'] = 1
df_test['CONTACTO_DIRECTO'] = np.where(df_test['INTENSIDAD_DIRECTA'] != 0, 1, 0)
df_test['PRIORIDAD_TELEFONO'] = df_test['PRIORIDAD_TELEFONO'].apply(lambda x: x if x in ['1', '2', '3', '4', '5+'] else 'NULL')

df_test.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
df_test.reset_index(drop=True, inplace=True)

df_test = sort_columns(df_test)

print(df_test.columns)
print(df_test.shape)
df_test.head(5)

In [12]:
fecha_actual = datetime.now()

def obtener_fecha_texto(fecha):
    mes = fecha.month
    año = fecha.year
    mes_nombre = calendar.month_abbr[mes].upper()[:3]  # ENE
    mes_año = f"{mes_nombre}{str(año)[2:]}"  # ENE24
    fecha_formato = f"{año}{str(mes).zfill(2)}"  # 202401
    return fecha_formato, mes_año

fecha_1, fecha_texto_1 = obtener_fecha_texto(fecha_actual)

fecha_un_mes_atras = fecha_actual - pd.DateOffset(months=1)
fecha_2, fecha_texto_2 = obtener_fecha_texto(fecha_un_mes_atras)

fecha_dos_meses_atras = fecha_actual - pd.DateOffset(months=2)
fecha_3, fecha_texto_3 = obtener_fecha_texto(fecha_dos_meses_atras)

In [ ]:
df_asignacion_1 = pd.read_excel(f'bases/asignacion/{fecha_1}/base_asignacion_{fecha_texto_1}_GEST.xlsx', sheet_name='BASE', dtype={'CONTRATO': str, 'CC': str})
print(df_asignacion_1.shape)

df_asignacion_2 = pd.read_excel(f'bases/asignacion/{fecha_2}/base_asignacion_{fecha_texto_2}_GEST.xlsx', sheet_name='BASE', dtype={'CONTRATO': str, 'CC': str})
print(df_asignacion_2.shape)

df_asignacion_3 = pd.read_excel(f'bases/asignacion/{fecha_3}/base_asignacion_{fecha_texto_3}_GEST.xlsx', sheet_name='BASE', dtype={'CONTRATO': str, 'CC': str})
print(df_asignacion_3.shape)

In [ ]:
#df_asignacion_clasa_1 = df_asignacion_1[df_asignacion_1['AGENCIA'] == 'CLASA MORA']
#print(df_asignacion_clasa_1.shape)
df_asignacion_rj_1 = df_asignacion_1[df_asignacion_1['AGENCIA'] == 'ASESCOM RJ']
print(df_asignacion_rj_1.shape)
df_asignacion_mornese_1 = df_asignacion_1[df_asignacion_1['AGENCIA'] == 'MORNESE MORA']
print(df_asignacion_mornese_1.shape)

#df_asignacion_clasa_2 = df_asignacion_2[df_asignacion_2['AGENCIA'] == 'CLASA MORA']
#print(df_asignacion_clasa_2.shape)
df_asignacion_rj_2 = df_asignacion_2[df_asignacion_2['AGENCIA'] == 'ASESCOM RJ']
print(df_asignacion_rj_2.shape)
df_asignacion_mornese_2 = df_asignacion_2[df_asignacion_2['AGENCIA'] == 'MORNESE MORA']
print(df_asignacion_mornese_2.shape)

#df_asignacion_clasa_3 = df_asignacion_3[df_asignacion_3['AGENCIA'] == 'CLASA MORA']
#print(df_asignacion_clasa_3.shape)
df_asignacion_rj_3 = df_asignacion_3[df_asignacion_3['AGENCIA'] == 'ASESCOM RJ']
print(df_asignacion_rj_3.shape)
df_asignacion_mornese_3 = df_asignacion_3[df_asignacion_3['AGENCIA'] == 'MORNESE MORA']
print(df_asignacion_mornese_3.shape)

In [23]:
def clean_columns_asignacion(df: pd.DataFrame) -> pd.DataFrame:
    df.rename(columns={
        'CC': 'COD_CENTRAL', 
        'CARTERA': 'MARCA', 
    }, inplace=True)
    
    df = df[df['MARCA'].isin(['BC', 'EXJ', 'KSTBC'])]
    
    return df[['COD_CENTRAL', 'CONTRATO', 'MARCA']]

df_asignacion_rj_1 = clean_columns_asignacion(df_asignacion_rj_1)
df_asignacion_rj_2 = clean_columns_asignacion(df_asignacion_rj_2)
df_asignacion_rj_3 = clean_columns_asignacion(df_asignacion_rj_3)

In [16]:
df_test_202412 = df_test[df_test['PERIODO'] == '202412']
df_test_202411 = df_test[df_test['PERIODO'] == '202411']
df_test_202410 = df_test[df_test['PERIODO'] == '202410']

In [ ]:
def cruzar_y_agregar(df_test_periodo, df_asignacion, periodo):
    df_resultado = df_test_periodo.copy()
    dias_unicos = sorted(df_test_periodo['DIA'].unique())
    contratos_acumulados = set()
    
    for dia in dias_unicos:
        print(f'Procesando día {dia} de {periodo}')
        df_test_acumulado = df_resultado[(df_resultado['DIA'] <= dia) & (df_resultado['INTENSIDAD_TOTAL'] == 1)]
        contratos_acumulados.update(df_test_acumulado['CONTRATO'].unique())
        print('Acumulado:', df_test_acumulado.shape)
        print('Dias únicos:', df_test_acumulado['DIA'].unique())
        print('Contratos Acumulados:', len(contratos_acumulados))
        
        df_cruce = pd.merge(df_test_acumulado, df_asignacion, on=['COD_CENTRAL', 'CONTRATO', 'MARCA'], how='outer', indicator=True)
        
        df_faltantes = df_cruce[df_cruce['_merge'] == 'right_only'].drop(columns=['_merge'])
        # Filtrar los contratos que ya están en el acumulado
        df_faltantes = df_faltantes[~df_faltantes['CONTRATO'].isin(contratos_acumulados)]
        print('Contratos Faltantes:', df_faltantes.shape[0])
        print('\n')
        
        df_faltantes['AGENCIA'] = 'ASESCOM RJ'
        df_faltantes['PERIODO'] = periodo
        df_faltantes['DIA'] = dia
        df_faltantes['MARCA'] = df_faltantes['MARCA'].fillna('NULL')
        df_faltantes['TRAMO'] = df_faltantes['TRAMO'].fillna('NULL')
        df_faltantes['TIPO_CONTACTO'] = df_faltantes['TIPO_CONTACTO'].fillna('SIN GESTION')
        df_faltantes['PDP'] = df_faltantes['PDP'].fillna(0)
        df_faltantes['TIPO_MARCADOR'] = df_faltantes['TIPO_MARCADOR'].fillna('SIN GESTION')
        df_faltantes['PRIORIDAD_TELEFONO'] = df_faltantes['PRIORIDAD_TELEFONO'].fillna('NULL')
        
        df_faltantes['INTENSIDAD_TOTAL'] = np.where(df_faltantes['INTENSIDAD_TOTAL'] == 1, 1, 0)
        df_faltantes['INTENSIDAD_DIRECTA'] = np.where(df_faltantes['TIPO_CONTACTO'] == 'CONTACTO DIRECTO', 1, 0)
        df_faltantes['COBERTURA'] = np.where(df_faltantes['INTENSIDAD_TOTAL'] == 1, 1, 0)
        df_faltantes['CONTACTO_DIRECTO'] = np.where(df_faltantes['INTENSIDAD_DIRECTA'] != 0, 1, 0)
        
        df_resultado = pd.concat([df_resultado, df_faltantes], ignore_index=True)
    
    return df_resultado

df_resultado_202412 = cruzar_y_agregar(df_test_202412, df_asignacion_rj_1, '202412')
df_resultado_202411 = cruzar_y_agregar(df_test_202411, df_asignacion_rj_2, '202411')
df_resultado_202410 = cruzar_y_agregar(df_test_202410, df_asignacion_rj_3, '202410')

In [18]:
df_test_1.to_excel('PRUEBA_DICIEMBRE.xlsx', index=False)

In [ ]:
dias_unicos = sorted(df_test_1['DIA'].unique())

df_final = pd.DataFrame()

for dia in dias_unicos:
    df_acumulado = df_test_1[df_test_1['DIA'] <= dia]
    print('Dia acumulado:', dia, df_acumulado.shape, df_acumulado['DIA'].unique())
    df_grouped = df_test_1.groupby(['AGENCIA', 'PERIODO', 'COD_CENTRAL', 'CONTRATO', 'TRAMO']).agg({
        'INTENSIDAD_TOTAL': 'sum', 
        'INTENSIDAD_DIRECTA': 'sum', 
        'COBERTURA': 'max',
        'CONTACTO_DIRECTO': 'max', 
        'PDP': 'max'
    }).reset_index()
    
    df_grouped['TASA_CIERRE'] = np.where(
        (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 1), 
        1, 
        np.where(
            (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 0), 
            0, 
            np.nan
            )
        )
    
    df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].apply(lambda x: int(x) if pd.notna(x) else x)
    
    df_grouped['INTENSIDAD_TOTAL'] = df_grouped['INTENSIDAD_TOTAL'].round(2)
    df_grouped['INTENSIDAD_DIRECTA'] = df_grouped['INTENSIDAD_DIRECTA'].round(2)
    df_grouped['COBERTURA'] = df_grouped['COBERTURA'].round(4)
    df_grouped['CONTACTO_DIRECTO'] = df_grouped['CONTACTO_DIRECTO'].round(4)
    df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].round(4)
    df_grouped['DIA'] = dia
    
    df_final = pd.concat([df_final, df_grouped])

df_final.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'TRAMO'], inplace=True)
df_final.reset_index(drop=True, inplace=True)

cols = ['AGENCIA', 'PERIODO', 'DIA', 'TRAMO', 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'COBERTURA', 'CONTACTO_DIRECTO', 'PDP']
df_final = df_final[cols]

df_final.to_excel('PRUEBA_DICIEMBRE.xlsx', index=False)

In [ ]:
df_final['TASA_CIERRE'].value_counts(dropna=False)

In [30]:
df_resultado_final = pd.concat([
    df_resultado_202412, 
    df_resultado_202411, 
    df_resultado_202410
], ignore_index=True)

df_resultado_final.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', 'MARCA', 'TRAMO', 'PRIORIDAD_TELEFONO', 'TIPO_MARCADOR'], inplace=True)
df_resultado_final.reset_index(drop=True, inplace=True)

In [ ]:
def group_data(df: pd.DataFrame, column: str) -> None:
    dias_unicos = sorted(df['DIA'].unique())
    
    df_final = pd.DataFrame()
    
    for dia in dias_unicos:
        df_acumulado = df[df['DIA'] <= dia]
        print('Dia acumulado:', dia, df_acumulado.shape, df_acumulado['DIA'].unique())
        
        df_grouped = df_acumulado.groupby(['AGENCIA', 'PERIODO', 'COD_CENTRAL', 'CONTRATO', column]).agg({
            'INTENSIDAD_TOTAL': 'sum', 
            'INTENSIDAD_DIRECTA': 'sum', 
            'COBERTURA': 'max',
            'CONTACTO_DIRECTO': 'max', 
            'PDP': 'max'
        }).reset_index()
        
        df_grouped['TASA_CIERRE'] = np.where(
            (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 1), 
            1, 
            np.where(
                (df_grouped['INTENSIDAD_DIRECTA'] > 0) & (df_grouped['PDP'] == 0), 
                0, 
                np.nan
                )
            )
        df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].apply(lambda x: int(x) if pd.notna(x) else x)
        
        df_grouped = df_grouped.groupby(['AGENCIA', 'PERIODO', column]).agg({
            'INTENSIDAD_TOTAL': 'mean', 
            'INTENSIDAD_DIRECTA': lambda x: x[x != 0].mean(), 
            'COBERTURA': 'mean',
            'CONTACTO_DIRECTO': lambda x: x.mean(skipna=True), 
            'TASA_CIERRE': lambda x: x.mean(skipna=True), 
        }).reset_index()
        
        df_grouped['INTENSIDAD_TOTAL'] = df_grouped['INTENSIDAD_TOTAL'].round(2)
        df_grouped['INTENSIDAD_DIRECTA'] = df_grouped['INTENSIDAD_DIRECTA'].round(2)
        df_grouped['COBERTURA'] = df_grouped['COBERTURA'].round(4)
        df_grouped['CONTACTO_DIRECTO'] = df_grouped['CONTACTO_DIRECTO'].round(4)
        df_grouped['TASA_CIERRE'] = df_grouped['TASA_CIERRE'].round(4)
        df_grouped['DIA'] = dia
        
        df_final = pd.concat([df_final, df_grouped])
    
    df_final.sort_values(by=['AGENCIA', 'PERIODO', 'DIA', column], inplace=True)
    df_final.reset_index(drop=True, inplace=True)
    
    cols = ['AGENCIA', 'PERIODO', 'DIA', column, 'INTENSIDAD_TOTAL', 'INTENSIDAD_DIRECTA', 'COBERTURA', 'CONTACTO_DIRECTO', 'TASA_CIERRE']
    df_final = df_final[cols]
    
    path = os.path.abspath(f'bases/gestiones/RJ/AGRUPACION_{column}.xlsx')
    df_final.to_excel(path, index=False)
    os.startfile(path)

group_data(df_resultado_final, 'MARCA')
group_data(df_resultado_final, 'TRAMO')
group_data(df_resultado_final, 'PRIORIDAD_TELEFONO')
group_data(df_resultado_final, 'TIPO_MARCADOR')